# Introduction

個別使用PTT creditcard版 2021年和2020年的語料做LDA，期望能夠透過非監督式的方法找出文章、句子的主題，語料處理如下：
1. 語料前處理
    - 刪除PTT標題分類（例如：[閒聊]、[發問]）
    - 刪除表情符號（字典 + unicode範圍）
    - 把多於空白刪除
    - 統一數字表示法（例如：1,000,000 -> 1000000）
    - 使用regex找出網址並刪除
    - 使用pysbd.Segmenter斷句
    - 使用CkipTagger斷詞
    - 切開過長句子
    - 合併過短句子
    - 把常見標點符號統一為全型
    - 刪除句子開頭或尾端的異常標點符號
    - 刪除 x%, x.x% 等回饋%數詞
    - 刪除純數字的詞
    
    
2. 語料過濾（句子）
    - 刪除OOV、特殊符號太多的句子
    - 刪除中文字太少的句子
    - 把標題包含「核卡」、「調額」的文章刪除
    

3. 語料過濾（單詞）
    - 刪除stopwords，參考：https://github.com/goto456/stopwords
    - 使用CkipTagger詞性標注後，刪除不是以下詞性的單詞，參考：https://github.com/ckiplab/ckiptagger/wiki/POS-Tags
        - Na: 普通名詞
        - Nb: 專有名詞
        - Nc: 地方詞
        - FW: 外文
        - VC: 動作及物動詞
    - 刪除只有一個字的單詞
    - 刪除「銀行名稱」單詞（若不刪除：則分群結果會有大量銀行名稱）
    
    
4. 訓練語料準備
    - 以文章為單位，準備n-gram語料（sliding window）
        - 目前n設定為1~5，也就是1-gram ~ 5-gram的語料會合併下去訓練
            - 實驗起來，比起用單一n-gram，更能在分群時找到更罕見的字作為主題
        - n-gram裡面必須包含至少一個銀行信用卡相關單詞，否則丟棄，參考：https://docs.google.com/spreadsheets/u/1/d/1o739ezU6qrFyw-QmbXBgu_8SlFjahOp-K5J-oddAXEM/edit#gid=1335809966

# Result

In [1]:
import sys
import warnings
from IPython.display import HTML
from os import getcwd
from os.path import dirname
from pyLDAvis import enable_notebook

enable_notebook()
warnings.filterwarnings('ignore')
sys.path.append(dirname(dirname(getcwd())))

from fintech_topic_detection.lda.visualization import LdaVisualization

In [2]:
EXP_ROOT = '/data/home/fintech-topic-detection/experiments'
EXP_DATE = '20211129'
EXP_NAME = 'creditcard2021-new'
EXP_PATH = f'{EXP_ROOT}/{EXP_DATE}/{EXP_NAME}'
ldavis = LdaVisualization(EXP_PATH, words_per_group=10, _lambda=0.6)

In [3]:
ldavis.show_groups()

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.031690  0.176110       1        1  14.218853
1      0.154309  0.065408       2        1  12.855895
7      0.010360 -0.078905       3        1  12.855095
0     -0.028585  0.071517       4        1  12.800855
3      0.160012 -0.065212       5        1  12.434326
4     -0.079636  0.035581       6        1  12.147061
6     -0.222635  0.024157       7        1  11.388919
5     -0.025514 -0.228656       8        1  11.298996, topic_info=    Term         Freq        Total Category  logprob  loglift
61    登錄  1098.000000  1098.000000  Default  30.0000  30.0000
51   PAY  1495.000000  1495.000000  Default  29.0000  29.0000
190  JCB   735.000000   735.000000  Default  28.0000  28.0000
667   店家   731.000000   731.000000  Default  27.0000  27.0000
178   上限  1404.000000  1404.000000  Default  26.0000  26.0000
..   ...          ...          ...      ...      ...      ...
368   條件    93.511107   245.728461   Topic8  -5.3092   1.2143
344   年費   109.826718   467.524713   Topic8  -5.1484   0.7319
178   上限   128.422382  1404.985871   Topic8  -4.9920  -0.2120
69    綁定    90.465278  1068.397836   Topic8  -5.3423  -0.2885
61    登錄    79.798599  1098.994697   Topic8  -5.4678  -0.4422

[561 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
989       3  0.981783  .COM
2730      3  0.955622    1M
1191      1  0.982539   2.0
3350      1  0.018365   24H
3350      5  0.973337   24H
...     ...       ...   ...
220       5  0.536751    點數
220       6  0.051189    點數
220       7  0.043876    點數
220       8  0.095065    點數
6575      8  0.969113   龜苓膏

[1574 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 8, 1, 4, 5, 7, 6])

In [4]:
ldavis.save_samples(min_prob=0.5, sample_nums=15, overwrite=True)

Group 1 keywords ['PAY', '客服', 'LINE', '五倍券', '新戶', '台灣', '申辦', '實體', 'ICASH', 'APPLE']
len(filtered) : sample_nums (103 : 15)
Group 2 keywords ['加碼', '街口', '感應', '大戶', '帳戶', '網購', '電商', '掃碼', '幣倍', '寰宇']
len(filtered) : sample_nums (185 : 15)
Group 3 keywords ['額度', '上限', '分期', '帳單', '計算', '保費', '蝦皮', '金額', '利率', '簡訊']
len(filtered) : sample_nums (159 : 15)
Group 4 keywords ['店家', '好食券', '綁定', 'OPEN', 'OP', '政府', '時間', '店員', '家人', '飯店']
len(filtered) : sample_nums (66 : 15)
Group 5 keywords ['登錄', 'JCB', '通路', 'KOKO', '紅利', '點數', 'PCHOME', 'GOGO', 'COMBO', '折抵']
len(filtered) : sample_nums (79 : 15)
Group 6 keywords ['家樂福', '儲值', '英雄聯盟', '盜刷', '全聯', '加辦', 'VISA', '中華電信', '鈦金', 'PLUS']
len(filtered) : sample_nums (32 : 15)
Group 7 keywords ['玫瑰', 'GIVING', '中油', '麥當勞', '白金', '整理', '小白', '參考', '版友', '自助']
len(filtered) : sample_nums (95 : 15)
Group 8 keywords ['PI', '御璽', '名額', '東奧', '一卡通', '橘子', '世界', '高鐵', '商務', 'UBEAR']
len(filtered) : sample_nums (37 : 15)
Samples are saved to /data

In [5]:
ldavis.show_keywords()

[['PAY', '客服', 'LINE', '五倍券', '新戶', '台灣', '申辦', '實體', 'ICASH', 'APPLE'],
 ['加碼', '街口', '感應', '大戶', '帳戶', '網購', '電商', '掃碼', '幣倍', '寰宇'],
 ['額度', '上限', '分期', '帳單', '計算', '保費', '蝦皮', '金額', '利率', '簡訊'],
 ['店家', '好食券', '綁定', 'OPEN', 'OP', '政府', '時間', '店員', '家人', '飯店'],
 ['登錄', 'JCB', '通路', 'KOKO', '紅利', '點數', 'PCHOME', 'GOGO', 'COMBO', '折抵'],
 ['家樂福', '儲值', '英雄聯盟', '盜刷', '全聯', '加辦', 'VISA', '中華電信', '鈦金', 'PLUS'],
 ['玫瑰', 'GIVING', '中油', '麥當勞', '白金', '整理', '小白', '參考', '版友', '自助'],
 ['PI', '御璽', '名額', '東奧', '一卡通', '橘子', '世界', '高鐵', '商務', 'UBEAR']]

In [6]:
HTML(ldavis.show_samples_table(2021, 1))

,title,text,keywords
1,[心得] 五倍券綁定玉山信用卡心得,另外玉山五倍券回饋的另一個好處是哪幾筆回饋都有顯示出來其他銀行的不知道，因為只能綁定一家，沒辦法跳船只能看其他板友有沒有要分享而已，用電腦的可以在這裡查詢，手機也可以，手機可以用這個查詢，可能該店家非好食券認列的業者？只能查稅籍登記看看，我平常也有人工記帳，所以也可以同時查詢哪些交易已入帳五倍券的交易哪幾筆已回饋同時備註上去，台灣Pay對我而言的問題是變成還要特別找能用的地方使用,五倍券|台灣
2,Re: [情報] 這夏刷台新 6%暑來發 - 8月,旅遊限定為全臺旅行社(含KLOOK、kkday)、航空公司(需透過國內外航空公司實體或官網購買機票之消費，且刷卡交易名稱需包含「航空」、「AIRLINE」始享回饋)、高鐵購票、臺鐵購票、國內租車、指定訂房網：Agoda、,LINE|實體
3,[心得] 2021Q1 信用卡回饋指定通路,、丹堤咖啡、怡客咖啡、湛盧，咖啡、黑沃咖啡、CAFE！N如使用多元支付綁定卡片交易且該筆消費於信用卡帳單明細有明確顯示商店名稱亦可計入回饋計算)限外幣)非台灣且非新台幣之一般消費(含實體商店及網路)或商店收單行為國外銀行之一,台灣|實體
4,Re: [情報] 五倍券登錄連結,五倍券登錄連結搭早安，感覺早上行政院網頁會炸：五倍券官網：：：以下是單純連結整理：信用卡/銀行類：：玉山：：：台新：：：華南：：：陽信：：：渣打：：：花旗：：：三信：：：國泰：：：星展中郵：：：富邦編輯好多次不知道為什麼就是不行)：：：元大：：：tml：匯豐：：：第一：：：日盛：：：安泰：：：永豐：：：台灣中小：：：土銀電子票券應該就沒那麼難搶再請大家自行去官網登錄嘍：大家加油(燦笑，我想玉山應該會是這次的最大的受惠者.,五倍券|台灣
5,[心得] OPEN POINT APP以台新卡線上支付不回饋！,台新一直說OPENPOINTAPP在他們系統看來就是統一超商消費，因此不會給予回饋。我說OPENPOINTAPP線上的消費怎麼會當作7-11消費？我根本沒有到實體門市啊！爭取了很久，最後才給我個案處理，說這次會給我回饋，但也說日後在OPENPOINTAPP上線上刷卡買東西都不會再給任何回饋。我說那你們要清楚寫在注意事項，客服說會再,客服|實體
6,[閒聊] 信用卡預借現金3萬用命拚了!請幫我加油!,信用卡預借現金3萬來把命拚！之前我用五倍券3天變三萬的運彩彩券沒有中贏魚我現在就是賭神2的賭神了已經跟台灣運彩仇笑癡對幹上了不是仇笑癡死就是我亡贏家就能上天堂,五倍券|台灣
7,Re: [討論] 玉山 Only 卡看來只能哩程卡功能了,看得出來目前就算是匯豐無限卡，：哩程回饋的%也跟玉山LV5換亞萬差不多。主要是你人在國外only換亞萬是還能接受的在台灣跟現金回饋的卡一比，就悲劇了以刷100元來說，能賺6哩亞萬，看起來是還不錯但是跟幣倍跟富邦J這種3%卡一比，有誰要用0.5元買1哩？,台灣
8,[情報] 全民拚振興，兆豐送百萬防疫金！,折抵與分期付款消費不得同時使用，若為紅利折抵交易，則以折抵後金額計算。(4)指定通路之定義I.百貨通路：含全台實體百貨業者，本行將依據特約商店向其請款銀行(收單機構)登錄之特約商店行業別代碼(MCCCode5311)與本行系統認定為基準(百貨業者限國別登記為台灣,台灣|實體
9,Re: [問題] 信用卡綁五倍券的帳單,兩者的回饋方式不一樣，雖然都是一銀那是行動支付範圍這邊的是直接綁一銀信用卡再五倍券台灣Pay綁一銀牽涉到一銀金融卡，所以回饋不是每月回饋,五倍券|台灣
10,[問題] 玉山好食券使用行動支付認列問題,其實我覺得綁定電子支付只能適用儲值點數消費那種，這樣才一致，實體券/信用卡/電子支付儲值，都是最終實際支出的終端節點，而該終端節點對於店家資訊才有全面掌握.而台灣Pay這種沒儲值綁信用卡的，就歸類為信用卡(AP，GP等亦同),台灣|實體


In [7]:
HTML(ldavis.show_samples_table(2021, 2))

,title,text,keywords
1,[閒聊] 2021下半年幣倍卡 回饋表格 基本問題釋疑,2021/07/01-12/31永豐幣倍卡新權益表格，單位都是台幣特選通路國內為6大行動支付街口被排除了，所以等於街口繳費部分確定沒特選加碼了，請留意)只是可以用手機或手錶來模擬感應刷卡,加碼|街口|感應
2,[建議] 消費習慣配合的信用卡是否合適,幣倍能打所有國內消費至少3%，但要有永豐外幣存款JU回饋LP點可能不合你要求，大戶才是針對Steam跟Udemp另外幾項可用UBear通包至於保費，看買哪家去申請專用的卡,大戶|幣倍
3,Re: [閒聊] 華南銀行東奧卡 誰給回饋 認列方式就是他說的算,所以新戶就不用急著解，45天內3筆888以上或累積3888以上的一般消費即可建議用一般消費認列的通路比較保險喔都是華南沒寫清楚，害某些人不相信台灣行動支付感應也算而森77啊，科科，感應2%是台灣行動支付給的掃碼2%是華南另外加碼的沒問題(其他配合銀行應該也有),加碼|感應|掃碼
4,[問題] 淘寶綁永豐JCB/幣倍的選擇？,淘寶綁永豐JCB/幣倍的選擇？原本之前買淘寶都是用大戶卡低調的賺8%不過今年的大戶卡指定通路已經不含國外網購了所以也必須換卡消費,大戶|網購|幣倍
5,Re: [情報] 新光OU搭信用卡 加碼10%額滿,線上成功申辦OU數位存款帳戶及新光信用卡(寰宇現金回饋卡或世界卡)除信用卡原產品權益優惠外消費加碼10%現金回饋。使用MyFamiPay支付代收款項每戶最高回饋100元/月,加碼|帳戶|寰宇
6,[情報] 彰銀6Pay筆筆享回饋My樂6%刷起來 5月登錄,面的PDF，有寫信用卡掃碼方式支付給2%等值現金之彰銀柴寶幣，這表示掃碼跟感應都有給2%返現吧。台灣PAY裡頭只顯示用帳戶支付的紅利點數，信用卡沒有帳戶、信用卡的2%回饋金上限100，且是分開計算。,感應|帳戶|掃碼
7,[閒聊] 永豐幣倍卡首刷禮,永豐幣倍卡首刷禮前陣子跟風辦了幣倍卡12月底核卡我有大戶帳戶但是一直沒辦大戶卡第一張永豐卡就是幣倍,大戶|帳戶|幣倍
8,[閒聊] 永豐幣倍排除街口後，繼續跌落幾階？ XD,"就算開放綁永豐卡，依照之前的街口邏輯應該也繞不過吧？"")2.速食店(摩斯)LINEPay部分可以低調繞過拿8%/9%倒是街口1月確認雖然無法拿特選加碼5%/6%所以有些實體卡無回饋或感應Pay無法低調繞過的部分",加碼|街口|感應
9,[情報] iLEO卡最新優惠,另外台灣PAY也加碼2%回饋(每期帳單回饋上限100元)但是要用掃碼的，感應的又被排除新戶綁定扣繳再加碼1%(每期帳單回饋上限100元)我這樣的理解有錯嗎？我記得上半年iLEO卡行動支付回饋好像沒那麼高？,加碼|感應|掃碼
10,Re: [情報] OU數存&世界卡/寰宇卡 享指定通路10%,OU數存&世界卡/寰宇卡，享指定通路10%去年9月中為了搬搬樂高利活存開了新光數位帳戶然後前幾天信用卡帳單來，發現四月份數位帳戶的加碼10%活動沒拿到昨晚以信件留言給客服，下午回電,加碼|帳戶|寰宇


In [8]:
HTML(ldavis.show_samples_table(2021, 3))

,title,text,keywords
1,[情報] 玉山銀行周末狂歡日(9-12月部分10%OP),獲得OPENPOINT回饋點數。(非玉山實體卡icash功能支付均不適用，如icashPay、手機icash2.0、手機icash2.0聯名卡等均不適用)。2.活動適用之通路門市需具接受icash2.0交易之機台設備，依業者現場公告為準。3.每月回饋上限以日曆月計算，每月1日重新計算累計回饋點數。,上限|計算
2,[問題] 花旗和匯豐的APP 哪個好一些？,我後來問了銀行有關刷卡通知的問題銀行說可以簡訊或email通知實體店面刷卡是3000元以上通知但網路購物、國外刷卡的通知不限金額,金額|簡訊
3,[情報] 繳信用卡費 用LINE Pay Money繳費享25點,LINEPayMoney繳納聯邦/台北富邦/新光銀行信用卡費活動內容活動期間使用LINEPayMoney繳納聯邦/台北富邦/新光銀行信用卡帳單，單筆繳款金額滿享LINEPOINTS75點回饋,帳單|金額
4,[情報] iLEO 卡指定行動支付消費最高2%回歸,iLEO卡指定行動支付消費最高2%回歸綁定帳戶扣繳行動支付消費最高2%現金回饋(每期帳單回饋上限250元)活動期間綁定本行帳戶扣繳iLEO信用卡卡費成功，台灣Pay(含台灣行動支付)消費享加碼2%現金回饋，以「指定行動支付」(含ApplePay、GooglePay、街口支付、LINEPay、全聯PX,上限|帳單
5,[心得] 花旗現金plus卡愚人節活動,17樓的t62大大，非共用額度花旗卡就是網銀一卡一帳號獨立帳單(老問題了，可以爬文一堆文)，除非有銀行戶頭才能一個網銀看全部卡，手機app最近才改成登入一個帳號可額度低，有其他家信用卡或是貸款嗎？有時其他家額度會,額度|帳單
6,Re: [問題] @gogo加碼3%問題,比方說2號結帳日帳單應繳金額3200元4號入帳一筆消費250元6號入帳一筆消費300元8號入帳一筆回饋-1000元,帳單|金額
7,Re: [情報] HSBC滙豐6/1起臨時調高額度消費不予回饋,別篇已經講過了，但被吵架淹沒，不好意思，我重講一次我和朋友分別致電匯豐客服，得到的回覆是，即使在額度內，匯豐如果認為你有獲利，額度內也是可以不給回饋溢繳保費前，我還問過客服可以獲得回饋嗎？當時還跟我,額度|保費
8,[心得] 2021下半年用卡整理,元大IPASS卡，加值滿3000回饋4%(自動加值2%回饋，滿3000下期首次自動加值回饋60)元大現賺IPASS2%一銀icash聯名卡&宜蘭認同icash卡，新卡icash自動加值10%OP(核卡後180天內)元大HelloKitty鑽金聯名卡，新戶自動加值5%(每期帳單回饋上限100),上限|帳單
9,Re: [情報] 永豐,永豐昨天未出帳可以看到回饋下來了只不過我是令人困惑的1元...29號結帳，所以撈12月份帳單的金額回饋？？,帳單|金額
10,"Re: [閒聊]雙狗 黑狗/飛狗,110年Q1加碼3%已入帳",4/22帳單剛剛滾完看app，雙狗vip似乎沒有含分期各期出帳湊滿單卡每月五千更正上次推文，雖然還沒收到電子帳單但登入網銀看帳單附加說明，雙狗vip分期應該算入，但flygoVip加碼的0.5%是下,分期|帳單


In [9]:
HTML(ldavis.show_samples_table(2021, 4))

,title,text,keywords
1,Re: [情報] icashpay 14大通路星期天10%回饋 速邁樂15%,icashpay14大通路星期天10%回饋，速邁樂15%這活動二月延續囉加碼回饋10%OPENPOINT，14大通路合併計算總回饋上限600萬點。、康是美、Semeur聖娜、聖德科斯、統一時代百貨台北店、夢時代購物中心(含統一時代,OPEN|OP
2,Fw: [情報] 上班不BLUE就用台新Pay OPENPOINT 20%大方送,上班不BLUE就用台新PayOPENPOINT20%大方送標題：[情報]上班不BLUE就用台新PayOPENPOINT20%大方送先講結論1.不需登錄(網頁沒寫，登錄活動區也沒有),OPEN|OP
3,[情報] 台塑週六OPEN POINT會員日活動,OPENPOINT會員於加油消費結帳時提供刷讀OPENPOINTAPP會員條碼，依當筆加油金額及支付方式，可享以下OPENPOINT點數回饋(1)以台塑聯名卡支付加油消費，按當筆加油發票金額OPENPOINT點數回饋9.9%。(2)以合作銀行信用卡支付加油消費，按當筆加油發票金額OPENPOINT點數回饋7.2%,OPEN|OP
4,[新聞] 好食券出包！ 被列入適用名單 多個店家不知情,會數位什麼，我們也不方便，我們一定要直接拿紙鈔才可以。」皮鞋店老闆也一頭霧水，明明是好食券，竟然還會在名單上，就連銀樓也被列入適用店家，名單上還有雜糧行，但根據地址走訪，卻是一間夾娃娃機店。,店家|好食券
5,"[問題] 店員多刷後,隔天刷退有此規定?",似乎不妥，這個只有店家才能處理了，有可能是事實也可能是劇本，言重了是人都會有失誤的時侯，有失誤也不代表沒腦子，感謝幫忙釋疑，輕鬆來看，沒簽名確實不可能請到款認真來看，家人在前一筆已在簽單上簽名了，所以要模仿也不是難事(猜)方法三，店員自刷一筆一仟多元，是否也是正確的？,店家|店員|家人
6,[情報] icashpay 14大通路星期天10%回饋,icashpay14大通路星期天10%回饋筆消費加碼回饋10%OPENPOINT，14大通路合併計算總回饋上限600萬點。活動通路1：MisterDonut統一多拿滋2、21風味館、21PLUS、星巴克、酷聖石、速邁樂加油站2、康是美、Semeur聖娜、聖德科斯、統一時代百貨台北店、夢時代購物中心(含統,OPEN|OP
7,[情報] 樂天分期滿一萬送600,看來是有預算沒花完combo微解封2%、jcb周末2%這樣還蠻威的只是要注意店家請款時間活動要在7/31前申辦分期,店家|時間
8,Re: [情報] 週五刷小小兵卡最高享１０%回饋,週五刷小小兵卡最高享10%回饋看起來登錄之後每週五都可以拿一次100元耶，活動區間7/16-10/29。：如果配合本來小小兵卡的1.234%，加上綁定TaiwanPay的2%，最高可以到13.234%，不知：道我理解有沒有錯誤？請問這個活動公布後到現在也差不多2週的時間了目前有人肉身測試過,綁定|時間
9,[情報] 玉山銀行周末狂歡日(9-12月部分10%OP),康是美指定日消費享10%OPENPOINT回饋2021/08/25-2021/09/24實體商店每卡、每戶及網路商店各30000元封頂熊熊「i」上你2021/09/01-2022/02/28持UBear粉卡，平日單筆若滿100麥當勞/康是美/星巴克也有10%3000封頂,OPEN|OP
10,[情報] 華南指定icash2.0聯名卡新卡禮,這活動有點坑，我也是前天在icashpay看到，華南官網的各信用卡裡面的最新活動跟專屬優惠裡完全找不到這活動....等看到時我3月多辦下的嘟嘟房聯名卡已經超過30天了只能再另外加辦OPENPOINT超級點數聯名卡再刷一次,OPEN|OP


In [10]:
HTML(ldavis.show_samples_table(2021, 5))

,title,text,keywords
1,Fw: [優惠] 全家 x 悠遊付 會員綁定悠遊付消費滿百(元)送千(點),利商店保留活動內容變更及最終解釋之權利。其他未盡事宜，悉依悠遊卡公司規定辦理。跟上次活動#1WVcdBp_(MobilePay)一樣全家點數基本上是1元累積1點，300點折抵1元,點數|折抵
2,[問題] 花旗現金回饋 plus 2% 鈦金卡打客服折抵,花旗現金回饋plus2%鈦金卡打客服折抵分享一下，原本用花旗app和桌機開網頁一直無法(或無合適選項折抵)，於是剛才有空便利用打客服的方式，後來用app看，成功折掉300元了。一開始會先聽一段語音，說信用卡和紅利什麼的不受影響，請用戶安心，接下來就是依照,紅利|折抵
3,[心得],年我的主力卡。目前還卡著400多萬的紅利點數6/8跟渣打同時提出申請，一樣上傳保價證明數張提出額度需求5.2M.然後就一直默默的等待中直到6/17我打電話進入客服詢問，我的審核結果,紅利|點數
4,Re: [討論] 花旗退出台灣,不能刷大白的都刷這張就好，沒有一堆限制，也沒超商刷卡沒紅利的限制。更新：剛剛管理室APP通知已到貨：※引述《imbmdj(in自修教室)》之銘言：：：花旗今宣布，將退出亞洲和EMEA(歐洲、中東和非洲)地區13個市場零售銀行事業，台：灣：：是其中之一。不知道新戶還可以拿AirPod嗎？：我於3/25號核卡：3/28刷pxpay儲值1.2w：昨天已收到耳機簡訊：等神腦出貨！還好還拿得到...：但是新戶8%還沒拿到：現在只有2%的點數,紅利|點數
5,Re: [情報] 花旗新戶辦現金回饋Plus卡送Airpods 2,花旗新戶辦現金回饋Plus卡送Airpods2借串問一下但我爬了一下文發現這張卡是回饋現金紅利，每300現金紅利才能折抵,紅利|折抵
6,[閒聊] 公益卡~最簡單做公益的方式,持卡享：新戶滿額送happygo1200點、首辦etag並儲值成功送100元儲值折抵金。合作金庫-心臟病兒童基金會認同卡方案：每筆刷卡消費提撥0.25%捐款給「中華民國心臟病兒童基金會」持卡享：終身免年費旅行平安險，白金卡享有3000萬元保障，普卡享有1250萬元。白金卡一般消費每滿25元，即可自動累積紅利積點一點。,紅利|折抵
7,Re: [問題] 花旗app沒更新,花旗app沒更新文章類別：1.問題：這是官網說app有紅利折抵功能：：應該要出現這樣！：：但我的app沒有這個選項：：也不知道哪時開始的，app也沒更新，想把點數用完，準備換卡了....忘了說，我是超級紅利回饋卡，不是現金回饋卡！：抱歉，應該算25點抵1元吧？我也是第一次用花旗的服務，狀況跟這位一樣，想說app怎麼都沒跑出折抵畫面推文裡面有人提到不適用才發覺這個問題，稍微研究一下終於搞懂了,紅利|點數|折抵
8,[情報] 中信中油卡 每周一線上儲值再加贈2%電子,中信中油卡，每周一線上儲值再加贈2%電子中信中油卡，每周一線上儲值再加贈2%電子油劵縮址：1.每周一線上儲值再加贈2%電子油劵2.中信中油卡線上儲值5%回饋3.中信中油卡紅利點數3倍，相當於0.8%回饋,紅利|點數
9,[問題] 中國信託中油聯名卡回饋,中國信託中油聯名卡回饋一直以來都有個疑問，中國信託這張聯名卡實際可以回饋多少9/20我使用中油PAY儲值了2000，贈送25元下次儲值的抵用券中信紅利點數回饋199點，算200點，在周六的時候每100點可折抵10元，算獲得20元,紅利|點數|折抵
10,[建議] 小白？老黑？漂白後首張卡請益,第一想申請的卡片]：台新GOGO第二想申請的卡片]：國泰KOKO其它]：永豐幣倍跟中信英雄聯盟也好想試試看年輕時不懂事曾經欠下卡債，後因開刀無法償還債務被強停，當時的債權銀行是花旗，所,KOKO|GOGO


In [11]:
HTML(ldavis.show_samples_table(2021, 6))

,title,text,keywords
1,[新聞] 晶片信用卡有複製盜刷技術？ 夫妻疑點多,紀錄，3天內在金飾店「周大福」、化妝品店「絲芙蘭」刷了7筆，但只有2筆、總額43萬7413元刷成。兩夫妻事後向交易成功的發卡銀行爭執帳款，銀行發覺有異，質疑「卡片都在身上怎會遭盜刷？」，要求兩人將信用卡繳回釐清，但周表示自己在中國廈門，周妻則說「人在台南,盜刷
2,[建議] 脫白失敗後續請益,脫白急不得，建議先等三個月後，再用二樓的方式脫白後(推家樂福)，再至少過三個月再挑戰永豐/台新雖然高回饋卡很誘人，但先從好過的卡開始養(累積信用)會比較理想,家樂福
3,[心得] 高鐵商務車廂升等、折價、現金回饋整理,樂現金回饋卡、VISA商務/商旅御璽卡當月免費升等1張(其餘請見下方表格)11.聯邦聯邦無限卡、聯邦世界卡、微風無限卡、大立無限卡、理財型無限卡、珍鑽悠遊無限卡、國旅卡、農金卡：正卡持卡人前月消費滿6萬元(含)以上可享當月免費升等每戶上限2張。,VISA
4,[心得] 花旗現金回饋PLUS鈦金卡轉卡（附卡面）,永豐銀行(360K)商務御璽卡(卓越)凱基銀行(360K)御璽卡(卓越)遠東商銀(320K)晶緻卡(卓越)台新銀行(300K)鈦金商務卡(卓越),鈦金
5,[閒聊] 聯邦微風無限卡年費直接減免,其實微風無限卡和他差一等級的卡差別沒有很多，頂多是點數多一倍和不限金額的停車，如果你沒那麼常去微風各館，御璽鈦金很夠用了，而且還有JCB悠遊卡10%可以搶給你年費減免只是希望你繼續持有而已,鈦金
6,[問題] 辦卡一直被打槍怎麼辦？,辦卡一直被打槍怎麼辦？板上高手大家好我老婆是中國籍，八年多前入台就在家父工廠投保至今，可是這兩三年辦理信用卡都被拒絕玉山家樂福卡,家樂福
7,[情報] 新光義大卡好好刷-8%回饋+新卡禮,一卡通就不是悠遊卡了，一卡通自動加值回饋本來就不多而且可以到8%想想要不要加辦注意事項有點不懂，應該是加油跟自動加值才有8%那關全家跟全聯什麼事？？,加辦
8,Fw: [情報] 悠遊付房屋稅繳稅5%回饋,本活動適用本國人參加，非本國人不適用。悠遊付帳戶儲值金回饋需依中華民國各類所得扣繳規定申報所得稅。依其規定，如全年所獲得的儲值金回饋超過新臺幣1000元，年度報稅時將計入個人其他所得。惟依財政部節能減碳所推動所得稅各類扣繳憑單免填發作業，悠遊卡公司將不寄送扣,儲值
9,Fw: [情報] 悠遊生日派對 累積滿2000元即可回饋100元,6.本公司加碼回饋需依中華民國各類所得扣繳規定申報所得稅。依其規定如全年所獲得的儲值金回饋超過新臺幣1000元，年度報稅時將計入個人其他所得。惟依財政部節能減碳所推動所得稅各類扣繳憑單免填發作業，本公司將不寄送扣(免)憑單予獲回饋之使用者。,儲值
10,[建議] 消費習慣配合的信用卡是否合適,已持有的卡片與額度]渣打現金回饋24萬中國信託商旅鈦金卡16萬玉山PI錢包16萬,鈦金


In [12]:
HTML(ldavis.show_samples_table(2021, 7))

,title,text,keywords
1,[問題] 聯邦理財型白金卡,聯邦理財型白金卡各位版友您好最近中華電信的寬頻繳費連台新的繳費系統都要拿掉回饋了,白金|版友
2,[討論] 關於置底文的整理與更新.,現在越來越多熱心的卡友開始也會幫各位整理常態性的消費也開始越來越多踴躍的整理文章出現.我覺得該是時候，結束這個任務.,整理
3,[心得] 中油pay + 中信中油聯名卡 省錢心得,ps第一次在週一儲值2000的1.25%回饋下次才能用沒錯！能省則省，比價我最厲害，還行啦，只要週一儲值，自助加油，不用算額度點數存到1500大概一年吧，再加一次人工加油即可，啥，我上面有放自助加油的優惠啊，哈哈，不過上週發現宜蘭員山鄉超偏僻的地方都有自助中油pay還算普及啦，我剛查這個優惠活動8/31就結束了，不限，只要下載中油pay上面會有適用的加油站，要刷15000，可以慢慢累積，這個不太懂，自助沒有中油VIP點數,中油|自助
4,[心得] 剪卡心得-中信兆豐聯邦一銀,4.一銀-綠活跟icash聯名卡先講結論：電話剪卡失敗爬版看到版友說有電話剪卡成功，也來試試看，問完基本資料客服馬上說卡片與悠遊卡、一卡通跟愛金卡公司聯結的，都不能電話剪卡，只能去分行或掛號寄卡，只好找時間去趟,版友
5,[問題] 中信中油卡 重複申請,中信中油卡，重複申請小弟前陣子申請中信中油聯名卡結果第一次申請選到白金卡看了消費回饋，御璽卡好一些些,中油|白金
6,[問題] 美運大白是信用卡嗎？,美運大白是信用卡嗎？我目前持有小白，正式名稱為美運信用白金卡，在聯徵上能看到這張且看得到對應額度以往爬文看到大白都是隱藏額度，本身也是稱為美運簽帳白金卡今天突然有個疑問是，大白到底是Debitcard還是Creditcard？,白金|小白
7,[情報] 中國信託中油卡權益通知,中國信託中油卡權益通知爬了板上沒看到這通知，應該沒OP。中油聯名卡到期轉換換成：中油聯名卡VISA御璽卡或白金卡,中油|白金
8,[問題] VISA白金卡的最大額度?,至於循環利率高低經詢問的結果是臺銀參考信用卡還款情況而決定的信用小白一開始一定是最高之後慢慢建立起信用後(每期都按時還款而不是只繳最低金額),小白|參考
9,Re: [心得] 2021上半年用卡整理,2021上半年用卡整理今天也整理了一下自己平常固定消費的2021信用卡最佳回饋，分享給大家參考如有錯誤也歡迎大家指證，感謝！以下回饋計算都是以已達成限定條件如：綁定數位帳戶、存款10萬以上等來計算),整理|參考
10,[討論] 花旗bye了之後之何去何從,接下來要辦什麼呢參考了一下板上資料針對花旗無腦2%(含保費)整理了一下資料1.富邦j卡,整理|參考


In [13]:
HTML(ldavis.show_samples_table(2021, 8))

,title,text,keywords
1,[心得] 高鐵商務車廂升等、折價、現金回饋整理,號座車票85折優惠2張。每戶每月最多2張優惠三)台北富邦尊御世界卡前一月份帳單「新增一般消費」達新臺幣6萬元，當月可享4日內(含交易日及乘車日)之標準車廂對號座票價88折之優惠，每月優惠上限2張。,世界
2,[情報] 收到國泰世界卡免年費邀請,收到國泰世界卡免年費邀請前一篇文到處調額後，可惜中信1M沒過剛剛收到國泰世華銀行的邀請簡訊ttps：//申辦，核卡30天內一般消費滿1萬元享1仟元百貨禮券，專為您款,世界
3,[情報] 臺灣企銀-藝FUN悠遊御璽卡,車燃料稅、交通違規罰款等)、信用卡買基金、悠遊卡自動加值金、一卡通自動加值金、年費、掛失、信用卡預借現金金額、信用卡各項手續費、循環信用利息、退貨交易、爭議款、違約金、代繳公用事業費(中華電信費、台電電費、台北市水費、台灣省水費)、學雜費、公務機關信用卡繳費平台、電子化繳費稅處理平台、來電分期每期入帳之款項、帳務,一卡通
4,[問題] 有人成功兌換過嗎？--花旗寰旅世界卡一泊,有人成功兌換過嗎？花旗寰旅世界卡一泊信箱收到行銷dm說有這個活動網址：一開始說預約網站1/4開放等到10點都沒連結公佈...打去客服問,世界
5,[分眾] 分眾活動兆豐卡滿萬額外2%,2.本行政府採購卡、商務卡及VISA金融卡不適用本活動。3.正、附卡消費合併計算，正、附卡合併每戶(以正卡人身分證字號歸戶計算)刷卡金回4.本活動消費金額為新臺幣計算，本活動交易消費日需介於活動期間內，且須於110年4月15日前消費店家已完成請款入帳者。若為紅利折抵交易，以紅利折抵後之交易金額計,商務
6,Re: [情報] HSBC匯豐現金回饋御璽卡終身免年費,HSBC匯豐現金回饋御璽卡終身免年費今天有去電客服詢問客服說因為最近推新卡新卡的優惠是電子/行動帳單免年費,御璽
7,Re: [情報] HSBC匯豐現金回饋御璽卡終身免年費,HSBC匯豐現金回饋御璽卡終身免年費這樣算是公告了嗎？年費不分新舊戶均享首年免年費，第二年起符合以下擇一條件即可終身免年費使用非紙本帳單(電子帳單或行動帳單),御璽
8,[情報] 中信行動購物節 消費最高贈威秀電影票2張,1.參加人於參加【中信行動購物節】(以下稱本活動)，即視同已詳閱並同意接受活動辦法及相關注意事項規定，且本活動需於中國信託活動登錄平台完成活動登錄。跨店消費(同一指定商電不論消費幾筆皆僅算一次，不累計)3.本活動資格限中信卡(含簽帳金融卡)持卡人，惟，排除英雄聯盟卡(信用卡、簽帳，金融卡)且於綁定付費場景消費始符合參加資格，排除LINEPay、街口、悠遊卡、一卡通，及信用卡自動加值等條碼支付及電子票證交易。4.滿額與否依本行登錄系統為。,一卡通
9,[情報] happycash有錢卡，指定通路享 8% 回饋金,"須當月登錄當月自動加值，每戶每月回饋上限50元Ref：-我是分隔線路人庚：四大電子票證，最近板上分享優惠，就獨漏""一卡通""這家機構路人己：我也這麼覺得，一卡通是不是獲利王！？油水給些點是要它的命嗎！？",一卡通
10,[問題] 目前收集長榮哩程最佳的信用卡是?,也想跟各位板友請益我的資訊是否正確1.玉山onlyLV5刷卡上限：約3571430海外11.218元1哩可得哩程：約318000哩年費：0元，現金回饋率(暫以0.32元/哩計算；不扣手續費1.5%)：2.85%2.國泰長榮極致御璽，刷卡上限：1000000,御璽
